In [1]:
import random

movies = ['Інтерстеллар', 'Начало', 'Джокер', 'Месники: Завершення', 'Темний лицар', 'Хороший рік']

def generate_user_movies():
    num_movies = random.randint(1, len(movies))
    return random.sample(movies, num_movies)

user_movie_data = {}
for i in range(1, 101):
    username = f'Користувач_{i}'
    user_movie_data[username] = generate_user_movies()

print(user_movie_data)

{'Користувач_1': ['Джокер', 'Хороший рік', 'Начало', 'Інтерстеллар', 'Месники: Завершення', 'Темний лицар'], 'Користувач_2': ['Начало', 'Темний лицар', 'Джокер', 'Хороший рік'], 'Користувач_3': ['Джокер', 'Начало', 'Інтерстеллар', 'Хороший рік'], 'Користувач_4': ['Начало', 'Джокер', 'Месники: Завершення', 'Інтерстеллар'], 'Користувач_5': ['Начало'], 'Користувач_6': ['Хороший рік', 'Начало'], 'Користувач_7': ['Джокер', 'Месники: Завершення', 'Хороший рік', 'Темний лицар', 'Інтерстеллар', 'Начало'], 'Користувач_8': ['Начало', 'Інтерстеллар'], 'Користувач_9': ['Темний лицар', 'Хороший рік'], 'Користувач_10': ['Начало', 'Інтерстеллар'], 'Користувач_11': ['Начало', 'Джокер', 'Інтерстеллар', 'Хороший рік', 'Темний лицар'], 'Користувач_12': ['Інтерстеллар', 'Месники: Завершення', 'Джокер'], 'Користувач_13': ['Начало'], 'Користувач_14': ['Джокер', 'Месники: Завершення'], 'Користувач_15': ['Темний лицар', 'Інтерстеллар', 'Начало'], 'Користувач_16': ['Хороший рік', 'Начало', 'Месники: Завершення